In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import statsmodels
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm #For Dickey-Fuller Test
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score,mean_squared_error
rcParams['figure.figsize'] = 15, 6

In [2]:
#Reading a csv file
df=pd.read_csv('shippable_revenue_forecasting.csv')
df.head()

,ABCIndicator,Month,ProductClassDesc,ProductLineDesc,SalesDistrictDesc,Shippable_$,Year
0,P,1,Underground,Enclosures & Drains,West Region,590.05,2017
1,P,1,Fiberglass Construction,Safety Products,Communication - West,0.00,2017
2,C,1,Fiberglass Construction,Safety Products,Communication - West,0.00,2017
3,A,1,Fiberglass Construction,Safety Products,Communication - West,435.77,2017
4,B,1,Fiberglass Construction,Safety Products,Communication - East,0.00,2017


In [3]:
train=df[1:round(len(df)*0.9)]  #here we can't use random train test split because this is time series data ,so we need values sequentially
train.shape

(31593, 7)

In [4]:
test=df[31593:]
test.shape

(3511, 7)

In [46]:
#Making a copy, to protect original data
train_original = train.copy()
test_original =test.copy()

In [47]:
train=train.loc[train.ProductClassDesc!='#'] #Removing # values
test = test.loc[test.ProductClassDesc!='#'] 

In [48]:
train.isnull().sum()# No Missing Values

ABCIndicator         0
Month                0
ProductClassDesc     0
ProductLineDesc      0
SalesDistrictDesc    0
Shippable_$          0
Year                 0
dtype: int64

In [49]:
train.ABCIndicator.value_counts()

C     8949
B     6770
A     6078
D     3695
1     3345
P     1326
##    1268
Name: ABCIndicator, dtype: int64

In [50]:
#Removing '##' values from ABCIndicator
train=train.loc[train.ABCIndicator!='##'] 
test = test.loc[test.ABCIndicator!='##'] 

In [51]:
df.groupby(['Year','Month']).sum()['Shippable_$']

Year  Month
2017  1        17306125.03
      2        15250667.97
      3        13606118.84
      4        11365164.37
      5        11936396.65
      6        11646166.27
      7        14436917.67
      8        14650194.96
      9        19142549.07
      10       16807334.24
      11       15045913.93
      12       11752345.95
2018  1        16490530.66
      2        14367265.73
      3        12966175.04
      4        16226271.05
      5        18107058.22
      6        15017874.28
      7        19964405.91
      8        18298135.40
      9        14591188.44
      10       16605596.90
      11       15099817.93
      12              0.00
2019  1               0.00
      2               0.00
      3               0.00
      4               0.00
      5               0.00
      6               0.00
      7               0.00
      8               0.00
      9               0.00
      10              0.00
      11              0.00
Name: Shippable_$, dtype: float64

In [52]:
train.ABCIndicator.value_counts()

C    8949
B    6770
A    6078
D    3695
1    3345
P    1326
Name: ABCIndicator, dtype: int64

In [53]:
train.ProductClassDesc.value_counts()

Traditional Tools                   1554
Pole Line Hardware                  1313
Distribution Connectors             1290
Resale                              1191
Utility Anchors                     1142
Distribution Arresters              1096
Underground                         1057
Transmission Connectors             1054
Substation Connectors               1000
Substation SC Arresters              991
URD Accessories                      934
PCORE Electric                       905
Cutouts                              900
Fiberglass Construction              894
Fuse Links                           848
Transmission Insulators              817
Distribution Insulators              809
Gloves and Sleeves                   761
Substation HC Arresters              760
Above Ground                         758
OVHD Switches                        748
Cutouts - Polymer                    709
URD Arresters                        577
Teleprotection (TPR)                 497
Substation Switc

In [54]:
# Now Lets try to predict on test data. But for that let's convert our features into numeric datatype
labelencoder = LabelEncoder()
train['ProductClassDesc']=labelencoder.fit_transform(train['ProductClassDesc'])
train['SalesDistrictDesc']=labelencoder.fit_transform(train['SalesDistrictDesc'])
train['ABCIndicator'] =labelencoder.fit_transform(train['ABCIndicator'])

In [55]:
dummies=pd.get_dummies(train['ProductLineDesc'])
train=pd.concat([train, dummies], axis=1)

In [56]:
del train['ProductLineDesc']

In [57]:
train.head()# Now All columns are numerical

,ABCIndicator,Month,ProductClassDesc,SalesDistrictDesc,Shippable_$,Year,Arresters,Bushings,Cable Accessories,Connectors,Construction,Enclosures & Drains,Insulators,Safety Products,Switching & Fusing,Utility Automation,Wire Management
1,5,1,21,5,0.00,2017,0,0,0,0,0,0,0,1,0,0,0
2,3,1,21,5,0.00,2017,0,0,0,0,0,0,0,1,0,0,0
3,1,1,21,5,435.77,2017,0,0,0,0,0,0,0,1,0,0,0
4,2,1,21,4,0.00,2017,0,0,0,0,0,0,0,1,0,0,0
5,1,1,21,4,1055.10,2017,0,0,0,0,0,0,0,1,0,0,0


In [18]:
#Now lets apply Linear Regression.First we will make a validation set by using traintest split.

In [20]:
Y=train['Shippable_$']
del train['Shippable_$']

In [21]:
X_train,X_valid,Y_train,Y_valid=train_test_split(train,Y,test_size=0.33,random_state=334)

In [23]:
regr=linear_model.LinearRegression()

In [24]:
regr.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [26]:
predictions=regr.predict(X_valid)

In [29]:
np.sqrt(mean_squared_error(Y_valid,predictions)) #very high rmse, this mean our model didn't perform well.

31644.29093897195

In [59]:
r2_score(Y_valid,predictions)

0.033267102225393086

In [60]:
from sklearn.ensemble import RandomForestRegressor

In [115]:
regr2 = RandomForestRegressor(max_depth=20, random_state=0,n_estimators=200,max_features='sqrt')

In [116]:
regr2.fit(X_train,Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [117]:
predictions=regr2.predict(X_valid)

In [118]:
np.sqrt(mean_squared_error(predictions,Y_valid))

23456.62751521539